# WebLLM with JavaScript Kernel

This notebook demonstrates how to load and use WebLLM directly in JavaScript, running in a Web Worker environment.

The JavaScript kernel allows us to work directly with WebLLM's native JavaScript API without Python wrappers.

**Note:** Since the JavaScript kernel doesn't support top-level `await`, we'll wrap async operations in immediately invoked async function expressions (IIFE).

In [ ]:
// Import WebLLM directly using dynamic import
console.log("Loading WebLLM...");

// Load WebLLM from CDN
async function loadWebLLM() {
    try {
        // Import WebLLM module
        const module = await import('https://cdn.jsdelivr.net/npm/@mlc-ai/web-llm@0.2.46/lib/index.min.js');
        
        // Make WebLLM available globally
        globalThis.WebLLM = module.webllm || module.default || module;
        
        console.log("✅ WebLLM loaded successfully!");
        console.log("Available WebLLM APIs:", Object.keys(globalThis.WebLLM));
        
        return true;
    } catch (error) {
        console.error("❌ Failed to load WebLLM:", error);
        return false;
    }
}

// Load WebLLM using IIFE (Immediately Invoked Function Expression)
(async () => {
    const success = await loadWebLLM();
    if (success) {
        console.log("🎉 Ready to proceed to next cell!");
    }
})();

In [ ]:
// Check if WebLLM is available
if (typeof globalThis.WebLLM === 'undefined') {
    console.log("⚠️ WebLLM not loaded. Please run the previous cell first.");
} else {
    console.log("🚀 WebLLM is ready!");
    console.log("Available methods:", Object.keys(globalThis.WebLLM));
    
    // Store WebLLM for easier access
    window.WebLLM = globalThis.WebLLM;
}

In [ ]:
// Create WebLLM engine and load model
let engine = null;

async function initializeWebLLM(modelId = "Llama-3.2-1B-Instruct-q4f16_1-MLC") {
    try {
        console.log("🔄 Initializing WebLLM engine...");
        console.log("⏳ This may take several minutes for the first download...");
        
        // Create the MLC engine
        engine = await globalThis.WebLLM.CreateMLCEngine(modelId);
        
        console.log("✅ WebLLM engine initialized successfully!");
        console.log("🎉 Ready to chat!");
        
        return true;
    } catch (error) {
        console.error("❌ Failed to initialize WebLLM:", error);
        return false;
    }
}

// Initialize the engine using IIFE
(async () => {
    const success = await initializeWebLLM();
    if (success) {
        console.log("✨ Model ready for use!");
    }
})();

In [ ]:
// Chat function
async function chat(message) {
    if (!engine) {
        return "❌ Engine not initialized. Please run the initialization cell first.";
    }
    
    try {
        console.log(`👤 User: ${message}`);
        
        const messages = [
            { role: "user", content: message }
        ];
        
        const response = await engine.chat.completions.create({
            messages: messages,
            max_tokens: 512,
            temperature: 0.7
        });
        
        const reply = response.choices[0].message.content;
        console.log(`🤖 Assistant: ${reply}`);
        
        return reply;
    } catch (error) {
        console.error("❌ Chat error:", error);
        return `Error: ${error.message}`;
    }
}

// Test chat using IIFE
(async () => {
    const response = await chat("Hello! What can you tell me about WebLLM?");
    console.log("Response received:", response);
})();

In [ ]:
// Text completion function
async function complete(prompt) {
    if (!engine) {
        return "❌ Engine not initialized. Please run the initialization cell first.";
    }
    
    try {
        console.log(`📝 Prompt: ${prompt}`);
        
        const response = await engine.completions.create({
            prompt: prompt,
            max_tokens: 256,
            temperature: 0.7
        });
        
        const completion = response.choices[0].text;
        console.log(`✨ Completion: ${completion}`);
        
        return completion;
    } catch (error) {
        console.error("❌ Completion error:", error);
        return `Error: ${error.message}`;
    }
}

// Test completion using IIFE
(async () => {
    const completion = await complete("The main advantages of running AI models in the browser are:");
    console.log("Completion received:", completion);
})();

In [ ]:
// Interactive demo with multiple examples
async function runDemo() {
    console.log("🎯 Running WebLLM Demo Session");
    console.log("=".repeat(50));

    const testMessages = [
        "Explain Pyodide in one sentence",
        "What makes WebLLM special for web development?", 
        "How does client-side AI benefit users?",
        "Tell me about the future of browser-based ML"
    ];

    // Run all test messages
    for (let i = 0; i < testMessages.length; i++) {
        const message = testMessages[i];
        console.log(`\n💬 Demo ${i + 1}:`);
        console.log("-".repeat(30));
        
        const response = await chat(message);
        
        // Add a small delay between requests
        await new Promise(resolve => setTimeout(resolve, 1000));
    }

    console.log("\n🎉 Demo complete!");
    console.log("💡 You can modify the testMessages array to try your own questions!");
}

// Run demo using IIFE
(async () => {
    await runDemo();
})();

In [ ]:
// Advanced WebLLM utilities
const WebLLMUtils = {
    // Get model information
    async getModelInfo() {
        if (!engine) return "Engine not initialized";
        
        try {
            console.log("📊 Model Information:");
            console.log("Engine:", engine);
            console.log("Engine type:", typeof engine);
            
            // Try to get more info if available
            if (engine.getModelInfo) {
                const info = await engine.getModelInfo();
                console.log("Model details:", info);
            }
            
            return "Model info logged to console";
        } catch (error) {
            return `Error getting model info: ${error.message}`;
        }
    },
    
    // Batch chat - send multiple messages
    async batchChat(messages) {
        if (!engine) return "Engine not initialized";
        
        const results = [];
        for (const message of messages) {
            try {
                const response = await chat(message);
                results.push({ message, response });
                
                // Small delay between requests
                await new Promise(resolve => setTimeout(resolve, 500));
            } catch (error) {
                results.push({ message, error: error.message });
            }
        }
        return results;
    },
    
    // Reset engine
    reset() {
        engine = null;
        console.log("🔄 Engine reset. Run initialization cell to restart.");
    }
};

// Test utility functions using IIFE
(async () => {
    console.log("🛠️ WebLLM Utilities loaded:");
    console.log("- WebLLMUtils.getModelInfo()");
    console.log("- WebLLMUtils.batchChat([messages])");
    console.log("- WebLLMUtils.reset()");
    
    // Get model info
    const info = await WebLLMUtils.getModelInfo();
    console.log("Info result:", info);
})();

In [ ]:
// Example: Custom chat with different parameters
async function customChat(message, options = {}) {
    if (!engine) {
        console.log("❌ Engine not initialized");
        return;
    }
    
    const defaultOptions = {
        max_tokens: 512,
        temperature: 0.7,
        top_p: 0.9
    };
    
    const chatOptions = { ...defaultOptions, ...options };
    
    try {
        const messages = [{ role: "user", content: message }];
        
        console.log(`🎛️ Chat with options:`, chatOptions);
        console.log(`👤 User: ${message}`);
        
        const response = await engine.chat.completions.create({
            messages: messages,
            ...chatOptions
        });
        
        const reply = response.choices[0].message.content;
        console.log(`🤖 Assistant: ${reply}`);
        
        return reply;
    } catch (error) {
        console.error("❌ Custom chat error:", error);
        return `Error: ${error.message}`;
    }
}

// Test custom chat with different temperatures
(async () => {
    console.log("🎨 Testing different chat parameters:");
    
    const testMessage = "Write a creative story about AI in the browser";
    
    // Creative response (high temperature)
    console.log("\n🔥 High creativity (temperature: 1.0):");
    await customChat(testMessage, { temperature: 1.0, max_tokens: 200 });
    
    // Balanced response (medium temperature)
    console.log("\n⚖️ Balanced (temperature: 0.7):");
    await customChat(testMessage, { temperature: 0.7, max_tokens: 200 });
    
    // Focused response (low temperature)
    console.log("\n🎯 Focused (temperature: 0.1):");
    await customChat(testMessage, { temperature: 0.1, max_tokens: 200 });
})();